In [48]:
import sys
sys.path.append("../")

import transforms as transform_module
import yaml
from types import SimpleNamespace

In [45]:
CONFIG_PATH = "../config.yaml"

with open(CONFIG_PATH) as file:
  config = SimpleNamespace(**yaml.safe_load(file))
  
  print(config)

namespace(train_dir='./data/raw/IRMAS_Training_Data', valid_dir='./data/raw/IRMAS_Validation_Data', preprocess={'PreprocessPipeline': {'target_sr': 22050}}, transforms={'Preemphasis': None, 'Spectrogram': {'sample_rate': 22050, 'n_mels': 128, 'hop_length': 160, 'n_fft': 800}, 'LogTransform': None, 'PadCutToLength': {'max_length': 1024}, 'Normalize': {'mean': -4.2677393, 'std': 9.1379948}}, signal_augments=None, spec_augments=None, batch_size=8, model_name='MIT/ast-finetuned-audioset-10-10-0.4593', loss={'FocalLoss': {'alpha': 0.25, 'gamma': 2}}, optimizer={'AdamW': {'weight_decay': 0.0001}}, learning_rates={'base_model': {'lr': 1e-08}, 'classifier': {'lr': 0.001}}, num_accum=2, verbose=True, EPOCHS=4, metrics=['hamming_score', 'zero_one_score', 'mean_average_precision', 'mean_f1_score'], scheduler={'OneCycleLR': {'pct_start': 0.3, 'anneal_strategy': 'cos'}}, early_stopping={'patience': 3, 'min_delta': 0.0001}, save_best_model=True)


In [ ]:
def init_objs(fn_dict, module):
    
    if fn_dict is None:
        return None

    transforms = []
    for transform in fn_dict.keys():
        fn = getattr(module, transform)
        fn_args = fn_dict[transform]
        
        if fn_args is None:
            transforms.append(fn())
        else:
            transforms.append(fn(**fn_args))
    
    return transforms

In [54]:


transforms = []
fn_dict = config.preprocess
for transform in fn_dict.keys():
    fn = getattr(transform_module, transform)
    fn_args = fn_dict[transform]
    
    transforms.append(fn(**fn_args))

In [55]:
config

namespace(train_dir='./data/raw/IRMAS_Training_Data',
          valid_dir='./data/raw/IRMAS_Validation_Data',
          preprocess={'PreprocessPipeline': {'target_sr': 22050}},
          transforms={'Preemphasis': None,
                      'Spectrogram': {'sample_rate': 22050,
                       'n_mels': 128,
                       'hop_length': 160,
                       'n_fft': 800},
                      'LogTransform': None,
                      'PadCutToLength': {'max_length': 1024},
                      'Normalize': {'mean': -4.2677393, 'std': 9.1379948}},
          signal_augments=None,
          spec_augments=None,
          batch_size=8,
          model_name='MIT/ast-finetuned-audioset-10-10-0.4593',
          loss={'FocalLoss': {'alpha': 0.25, 'gamma': 2}},
          optimizer={'AdamW': {'weight_decay': 0.0001}},
          learning_rates={'base_model': {'lr': 1e-08},
                          'classifier': {'lr': 0.001}},
          num_accum=2,
          verbose=Tr

In [4]:
run = wandb.init(config=config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: k-pintaric. Use `wandb login --relogin` to force relogin


In [5]:
config = SimpleNamespace(**wandb.config.parameters)

In [10]:
config

namespace(train_dir={'value': './data/processed/all_sync/IRMAS_Training_Data'},
          valid_dir={'value': './data/raw/IRMAS_Validation_Data'},
          preprocess={'PreprocessPipeline': {'parameters': {'target_sr': {'value': 22050}}}},
          transforms={'Preemphasis': {'value': None},
                      'Spectrogram': {'parameters': {'sample_rate': {'value': 22050},
                        'n_mels': {'values': [128, 96, 64]},
                        'hop_length': {'values': [64, 128, 256, 512]},
                        'n_fft': {'values': [512, 1024, 2048]}}},
                      'LogTransform': {'value': None},
                      'PadCutToLength': {'parameters': {'max_length': {'value': 1024}}},
                      'Normalize': {'parameters': {'mean': {'value': -4.2677393},
                        'std': {'value': 9.1379948}}}},
          batch_size={'value': 8},
          loss={'FocalLoss': {'parameters': {'alpha': {'values': [-1,
                    0.25,
        

In [22]:
config.learning_rates

{'base_model': {'parameters': {'lr': {'distribution': 'log_uniform_values',
    'min': 1e-08,
    'max': 1e-05}}},
 'classifier': {'parameters': {'lr': {'distribution': 'log_uniform_values',
    'min': 0.0001,
    'max': 0.1}}}}

In [21]:
config.preprocess

{'PreprocessPipeline': {'parameters': {'target_sr': {'value': 22050}}}}

In [17]:
from torchvision.transforms import Compose

In [21]:
def init_obj(fn_dict, module, *args, **kwargs):
    
    if isinstance(fn_dict, dict):
        fn_name = list(fn_dict.keys())[0]
    if isinstance(fn_dict, str):
        fn_name = fn_dict
    
    fn = getattr(module, fn_name)
    fn_args = fn_dict[fn_name].get("parameters")
    
    if fn_args is not None:
        assert all([k not in fn_args for k in kwargs])
        fn_args.update(kwargs)
        
        return fn(*args, **fn_args)
    else:
        return fn(*args, **kwargs)

In [33]:
def init_transforms(config, module_name):
    
    if config is None:
        return None

    transforms = []
    for transform in config.keys():
        print(transform)
        fn = getattr(transform_module, transform)
        transforms.append(fn(config[transform]["parameters"]))
    
    return Compose(transforms)

In [56]:
def init_objs(fn_dict, module):
    
    if fn_dict is None:
        return None

    transforms = []
    for transform in fn_dict.keys():
        fn = getattr(module, transform)
        fn_args = fn_dict[transform]
        
        if fn_args is None:
            transforms.append(fn())
        else:
            fn_args = fn_args.get("parameters")
            transforms.append(fn(**fn_args))
    
    return transforms

In [76]:
fn_dict = config.preprocess

transforms = []
for transform in fn_dict.keys():
    fn = getattr(transform_module, transform)
    fn_args = fn_dict[transform]

    if fn_args is None:
            transforms.append(fn())
    else:
        fn_args = fn_args.get("parameters")
        transforms.append(fn(**fn_args))

In [86]:
init_obj(config.preprocess, transform_module)

PreprocessPipeline()

In [89]:
config.signal_augments is None

True

In [51]:
config.optimizer.get("parameters")

In [34]:
import transforms as transform_module
init_transforms(config.transforms, transforms)

Preemphasis


TypeError: 'NoneType' object is not subscriptable

In [16]:
import torch.optim as optim

In [15]:
from utils import init_obj

In [20]:
import loss

In [22]:
import models

In [24]:
model = models.RNN(128, 64, 3, 11)

In [30]:
opt = init_obj(config.optimizer, optim, model.parameters())

In [30]:

def unflatten_dot(dictionary):
    resultDict = dict()
    for key, value in dictionary.items():
        parts = key.split(".")
        d = resultDict
        for part in parts[:-1]:
            if part not in d:
                d[part] = dict()
            d = d[part]
        d[parts[-1]] = value
    return resultDict

In [32]:
config = unflatten_dot("../sweep_config.yaml")

AttributeError: 'str' object has no attribute 'items'

In [39]:
CONFIG_PATH = "../config.yaml"

with open(CONFIG_PATH) as file:
  config = yaml.safe_load(file)
  
  print(config)

{'train_dir': './data/raw/IRMAS_Training_Data', 'valid_dir': './data/raw/IRMAS_Validation_Data', 'preprocess': {'PreprocessPipeline': {'target_sr': 22050}}, 'transforms': {'Preemphasis': None, 'Spectrogram': {'sample_rate': 22050, 'n_mels': 128, 'hop_length': 160, 'n_fft': 800}, 'LogTransform': None, 'PadCutToLength': {'max_length': 1024}, 'Normalize': {'mean': -4.2677393, 'std': 9.1379948}}, 'signal_augments': None, 'spec_augments': None, 'batch_size': 8, 'model_name': 'MIT/ast-finetuned-audioset-10-10-0.4593', 'loss': {'FocalLoss': {'alpha': 0.25, 'gamma': 2}}, 'optimizer': {'AdamW': {'weight_decay': 0.0001}}, 'learning_rates': {'base_model': {'lr': 1e-08}, 'classifier': {'lr': 0.001}}, 'num_accum': 2, 'verbose': True, 'EPOCHS': 4, 'metrics': ['hamming_score', 'zero_one_score', 'mean_average_precision', 'mean_f1_score'], 'scheduler': {'OneCycleLR': {'pct_start': 0.3, 'anneal_strategy': 'cos'}}, 'early_stopping': {'patience': 3, 'min_delta': 0.0001}, 'save_best_model': True}


In [42]:
config["transforms"]["LogTransform"]

In [34]:
unflatten_dot(config["parameters"])

{'train_dir': {'value': './data/processed/all_sync/IRMAS_Training_Data'},
 'valid_dir': {'value': './data/raw/IRMAS_Validation_Data'},
 'preprocess': {'PreprocessPipeline': {'parameters': {'target_sr': {'value': 22050}}}},
 'transforms': {'Preemphasis': {'value': None},
  'Spectrogram': {'parameters': {'sample_rate': {'value': 22050},
    'n_mels': {'values': [128, 96, 64]},
    'hop_length': {'values': [64, 128, 256, 512]},
    'n_fft': {'values': [512, 1024, 2048]}}},
  'LogTransform': {'value': None},
  'PadCutToLength': {'parameters': {'max_length': {'value': 1024}}},
  'Normalize': {'parameters': {'mean': {'value': -4.2677393},
    'std': {'value': 9.1379948}}}},
 'batch_size': {'value': 8},
 'loss': {'FocalLoss': {'parameters': {'alpha': {'values': [-1,
      0.25,
      0.5,
      0.75]},
    'gamma': {'values': [1, 2, 3]}}}},
 'optimizer': {'AdamW': {'parameters': {'weight_decay': {'distribution': 'log_uniform_values',
     'min': 1e-07,
     'max': 0.1}}}},
 'learning_rates': 